# Bloque 4

In [1]:
from google.cloud import aiplatform_v1
import streamlit as st 
from google.cloud import bigquery
from langchain_google_vertexai.llms import VertexAI
from langchain.prompts import PromptTemplate
import vertexai
import os
from langchain_google_vertexai import VertexAIEmbeddings

# Definir una función bot_answers

### Esta función debe encapsular:

- Obtenención de la descripción del producto y su embedding
- Obtención producto más similar y recomendarlo

In [2]:
# Configuración para el índice desplegado
API_ENDPOINT="2069128544.us-central1-1043238928011.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/1043238928011/locations/us-central1/indexEndpoints/6459098649556156416"
DEPLOYED_INDEX_ID="products_data_index_civica"

# Cliente Vector Search
client_options = {"api_endpoint": API_ENDPOINT}
vector_search_client = aiplatform_v1.MatchServiceClient(client_options=client_options)

# Cliente BigQuery
client = bigquery.Client()

# Obtención embeddings
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")   ### Utilizamos la clase VertexAIEmbeddings y el método embed_query()

# Configuración LLM (Gemini)
llm = VertexAI(model_name="gemini-pro", temperature=0.1)

In [3]:
embedding_template = """Eres el primer eslabón de una cadena que conformará un bot ecommerce y necesito en este primer paso recoger únicamente la información referente al producto que nos da el usuario.

El objetivo es que a partir de tu output se generará un embedding para buscar este producto que pide el usuario en nuestra base de datos vectorial (por tanto no te inventes nada).

Ejemplo: "Quiero unas gafas de sol negras porque estamos en verano", tu output debe ser "Gafas de sol negras".

La pregunta introducida por el usuario es: "{question}"

Output para generar el embedding: """

In [4]:
prompt_embedding_template = PromptTemplate.from_template(embedding_template)

In [5]:
embedding_chain = prompt_embedding_template | llm

In [12]:
template = """Eres un bot encargado de responder y asesorar sobre productos de
una tienda en linea (ecommerce). Por lo tanto, se respetuoso y trata cordialmente a las personas.
Si la pregunta suya no tiene nada que ver con asesoramiento de productos, dile que ese no es tu trabajo.

Pregunta: {question}

El resultado más similar encontrado en la base de datos de productos es el siguiente:
{most_similar}

Dile al cliente nuestra recomendación según lo que ha preguntado informándole de todo lo posible para que pueda decidirse en su compra como nombre del artículo o precio.
Responde en el mismo idioma en el que te pregunten.

Respuesta: """

In [7]:
prompt_template = PromptTemplate.from_template(template)

In [8]:
prompt = "Hola, qué tal, quiero unos vaqueros azules de talla pequeña porque tengo una fiesta mañana"

In [9]:
chain = prompt_template | llm

In [10]:
def bot_answer(prompt):

    # Ejecutamos la cadena chain_embedding para obtener la descripción del producto en el que está interesado el usuario utilizando Gemini
    prompt_embedding = embedding_chain.invoke({"question": prompt})
    print(f"Texto para embedding: {prompt_embedding}")
    
    # Construcción FindNeighborsRequest object
    datapoint = aiplatform_v1.IndexDatapoint(
      feature_vector=embeddings.embed_query(prompt_embedding)
    )
    query = aiplatform_v1.FindNeighborsRequest.Query(
      datapoint=datapoint,     
      neighbor_count=3  # Número de vecinos cercanos a recuperar
    )
    request = aiplatform_v1.FindNeighborsRequest(
      index_endpoint=INDEX_ENDPOINT,
      deployed_index_id=DEPLOYED_INDEX_ID,
      queries=[query],
      return_full_datapoint=False,
    )

    # Ejecutamos request y obtenemos el id en BBDD del producto más similar a la descripción pedida por el usuario
    response = vector_search_client.find_neighbors(request)
    neighbors = response.nearest_neighbors[0].neighbors 
    most_similar = neighbors[0].datapoint.datapoint_id
    
    # Buscamos el ID en BigQuery para obtener el resto de información
    sql = f"""
    SELECT category, brand, name, retail_price as price, department as gender
    FROM ia-ugr.ecommerce.products
    WHERE ID = {most_similar}
    ;
    """
    sql_result = client.query(sql).to_dataframe()
    print(f"Resultado SQL es: {sql_result}")

    # Ejecutamos la cadena chain para obtener la recomendación para el usuario utilizando la información recuparada de BigQuery y Gemini
    bot_result = chain.invoke({"question": prompt, "most_similar": sql_result})

    return bot_result

In [11]:
full_response = bot_answer(prompt)

Texto para embedding: "Vaqueros azules talla pequeña" 

Resultado SQL es:   category   brand                                               name  price  \
0   Shorts  Hurley  Lowrider Juniors 2.5 Inch Shorts in Ice Blue b...   24.0   

  gender  
0  Women  
